## Term Freq * Inverse Doc Freq Analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize

### Load Data

In [2]:
# Read in the Telematics Trip Data
bucket = 's3://cortex-ni-wc-claims'
file = 'cleantxt_df.csv'
# col_type = {'trip_id': int, 'avg_speed': float, 'max_speed': float, 'avg_speed_limit': float, 
#             'sample_count':int, 'driver_name':object}
# df = pd.read_csv(f'{bucket}/{file}', dtype = col_type)

df = pd.read_csv(f'{bucket}/{file}')
print('Data shape: ', df.shape)

Data shape:  (544529, 3)


In [3]:
df.head()

accident_txt_cleaned injury_txt_cleaned  \
0  assoicate injuring restriction associate said ...            unknown   
1  working lightbulb aisle rochelle noticed ache ...            unknown   
2                           causeunspecified medical            unknown   
3  walked associate saying couldnt see walked eye...            unknown   
4                josh taking item microwave fell arm            unknown   

   deEscInd3000  
0             1  
1             0  
2             1  
3             1  
4             1

In [4]:
# Additional cleaning

#stop words  not included in nltk "english"
extra_stopwords = set(['left', 'right', 'employee', 'emp', 'ee', 'associate', 'iw']) 
# what is 'iw' ???
# !!!may need to revise the stop words list later

def rm_stopwords(text):
    """
        input: a string
        
        return: cleaned string
    """
    word_tokens = word_tokenize(text) 
    word_tokens = [w for w in word_tokens if not w in extra_stopwords ] # delete stopwords from text
        
    text = " ".join(word_tokens)
    return text

### Overall Data Analysis

In [5]:
import time
start_time = time.time()

accident_cleaned = df['accident_txt_cleaned']
injury_cleaned = df['injury_txt_cleaned']

accident_cleaned= accident_cleaned.fillna('')
injury_cleaned = injury_cleaned.fillna('')

accident_cleaned= accident_cleaned.apply(rm_stopwords)
injury_cleaned= injury_cleaned.apply(rm_stopwords)

print("--- %s seconds ---" % (time.time() - start_time))

--- 125.04878807067871 seconds ---


#### Unigram Analysis

In [6]:
# creating the feature matrix 
from sklearn.feature_extraction.text import TfidfVectorizer
import time

start_time = time.time()

acct_vectorizer1 = TfidfVectorizer(max_features=1000)
inj_vectorizer1 = TfidfVectorizer(max_features=1000)
acct_tfidf1 = acct_vectorizer1.fit_transform(accident_cleaned).toarray()
inj_tfidf1 = inj_vectorizer1.fit_transform(injury_cleaned).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 14.679110288619995 seconds ---


In [7]:
acct_features1 = acct_vectorizer1.get_feature_names()
acct_features1

['10',
 '12',
 '15',
 '20',
 '2nd',
 '3rd',
 '911',
 'abdomen',
 'abdominal',
 'ac',
 'accident',
 'accidentally',
 'accidently',
 'across',
 'activity',
 'adjusting',
 'aggravated',
 'ago',
 'air',
 'aisle',
 'alleged',
 'allegedly',
 'alleges',
 'allergic',
 'along',
 'also',
 'altercation',
 'ambulance',
 'ankle',
 'another',
 'anxiety',
 'apart',
 'approached',
 'approx',
 'approximately',
 'area',
 'arm',
 'around',
 'asbestos',
 'asked',
 'assisting',
 'assoc',
 'assocaite',
 'attack',
 'attacked',
 'attempted',
 'attempting',
 'attention',
 'attic',
 'auto',
 'avoid',
 'away',
 'back',
 'backed',
 'backing',
 'backward',
 'backwards',
 'bad',
 'bag',
 'balance',
 'ball',
 'band',
 'banding',
 'bar',
 'barrel',
 'base',
 'basket',
 'bathroom',
 'battery',
 'bay',
 'beam',
 'became',
 'bed',
 'bee',
 'began',
 'behind',
 'belief',
 'belt',
 'bend',
 'bending',
 'bent',
 'bicep',
 'big',
 'bilateral',
 'bin',
 'bit',
 'bite',
 'bitten',
 'black',
 'blade',
 'bleeding',
 'blew',
 'b

In [8]:
acct_freq1 = np.sum(acct_tfidf1, axis=0)
acct_tfidf_freq1 = pd.DataFrame({'acct_features1': acct_features1, 
                               'freq1': acct_freq1}) 
acct_tfidf_freq1.sort_values(by=['freq1'], ascending = False). head(20)

acct_features1         freq1
52            back  18720.746766
300           fell  18291.502461
574           pain  17362.008927
301           felt  15347.766831
430             iw  14303.862503
780        slipped  12413.576962
375           hand  11739.165912
441           knee  10029.621658
470        lifting   9915.902961
962        walking   9890.372942
397            hit   9799.329504
755       shoulder   9137.283431
103            box   9026.263544
381           head   8530.547136
944        unknown   8482.609625
306         finger   8300.393157
320           foot   8280.926856
494          lower   8028.062083
155        causing   7969.694227
220            cut   7283.951162

In [9]:
inj_features1 = inj_vectorizer1.get_feature_names()
inj_features1

['10',
 '12',
 '1st',
 '2nd',
 '3rd',
 '40',
 '4th',
 '52',
 '5th',
 'abdomen',
 'abdominal',
 'abrasion',
 'accident',
 'ache',
 'achilles',
 'aching',
 'acl',
 'across',
 'acute',
 'ago',
 'air',
 'aisle',
 'alleged',
 'alleges',
 'allergic',
 'along',
 'also',
 'ambulance',
 'amputation',
 'ankle',
 'another',
 'anxiety',
 'approx',
 'area',
 'arm',
 'armshoulder',
 'around',
 'arthritis',
 'asbestos',
 'asbestosis',
 'asked',
 'assisting',
 'assoc',
 'asthma',
 'attack',
 'attempting',
 'auto',
 'available',
 'avulsion',
 'away',
 'back',
 'backed',
 'backwards',
 'bad',
 'bag',
 'bakery',
 'balance',
 'ball',
 'bar',
 'base',
 'basket',
 'bathroom',
 'battery',
 'beam',
 'became',
 'bed',
 'bee',
 'began',
 'behind',
 'belief',
 'belly',
 'belt',
 'bend',
 'bending',
 'bent',
 'bicep',
 'big',
 'bilateral',
 'bin',
 'bit',
 'bite',
 'black',
 'blacked',
 'blade',
 'bleed',
 'bleeding',
 'blister',
 'block',
 'blood',
 'blunt',
 'blurred',
 'board',
 'bodily',
 'body',
 'bolt',
 'b

In [10]:
inj_freq1 = np.sum(inj_tfidf1, axis=0)
inj_tfidf_freq1 = pd.DataFrame({'inj_features1': inj_features1, 
                               'freq1': inj_freq1}) 
inj_tfidf_freq1.sort_values(by=['freq1'], ascending = False). head(20)

inj_features1          freq1
948                 unknown  142140.377388
805  sprainstraintearjammed   28040.903292
602                    pain   25397.355096
839                  strain   22632.459898
50                     back   21872.252007
111         bruisecontusion   20104.355701
514                   lower   15271.876524
467                    knee   14939.269573
765                shoulder   14222.966535
388                    hand   12134.915792
200               contusion   12119.513087
443                  injury   10887.776498
996                   wrist   10876.384242
475              laceration    9604.159008
331                  finger    9080.723157
393                    head    8822.023144
477    lacerationcutscratch    8702.206757
29                    ankle    8399.046471
802                  sprain    7612.243094
34                      arm    7186.113475

In [11]:
inj_tfidf_freq1.sort_values(by=['freq1'], ascending = True). head(20)

inj_features1      freq1
439        inhale  57.023312
691         reach  59.079583
32         approx  59.542617
814         start  60.290360
40          asked  60.608237
321          fast  61.234295
726           row  61.491351
637        placed  61.676692
178          coal  61.949166
894          thru  62.611819
633        pinned  62.851704
210       couldnt  63.050005
157        cheese  64.288429
294      entering  64.756073
720          rock  65.205001
459           jam  65.248603
585       occured  65.265533
508       looking  65.284078
123           bus  65.501481
788          soda  65.965916

#### Bigram Analysis

In [12]:
start_time = time.time()

acct_vectorizer2 = TfidfVectorizer(max_features=500,
                                  ngram_range=(2, 2)) # look at two gram phrase only
inj_vectorizer2 = TfidfVectorizer(max_features=500,
                                 ngram_range=(2, 2))
acct_tfidf2 = acct_vectorizer2.fit_transform(accident_cleaned).toarray()
inj_tfidf2 = inj_vectorizer2.fit_transform(injury_cleaned).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 19.72451949119568 seconds ---


In [13]:
acct_features2 = acct_vectorizer2.get_feature_names()
acct_features2

['alleges injury',
 'allergic reaction',
 'ankle fell',
 'another vehicle',
 'arm hand',
 'arm shoulder',
 'assisting customer',
 'auto accident',
 'back area',
 'back felt',
 'back head',
 'back hit',
 'back hurt',
 'back hurting',
 'back injury',
 'back knee',
 'back leg',
 'back lifting',
 'back neck',
 'back pain',
 'back shoulder',
 'back side',
 'back started',
 'back strain',
 'back truck',
 'balance fell',
 'bed felt',
 'began feel',
 'began hurt',
 'bending pick',
 'bent pick',
 'big toe',
 'bilateral hearing',
 'black ice',
 'body part',
 'box box',
 'box cutter',
 'box fell',
 'box felt',
 'box tile',
 'break room',
 'bucket paint',
 'bumped head',
 'came back',
 'came contact',
 'came loose',
 'came work',
 'car accident',
 'car felt',
 'car slipped',
 'carpal tunnel',
 'carrying box',
 'carrying ladder',
 'cart felt',
 'cart hit',
 'caught foot',
 'caught hand',
 'caused injury',
 'caused pain',
 'causing contusion',
 'causing cut',
 'causing fall',
 'causing injury',
 'ca

In [14]:
acct_freq2 = np.sum(acct_tfidf2, axis=0)
acct_tfidf_freq2 = pd.DataFrame({'acct_features2': acct_features2, 
                               'freq2': acct_freq2}) 
acct_tfidf_freq2.sort_values(by=['freq2'], ascending = False). head(20)

acct_features2         freq2
140       felt pain  12867.703573
283      lower back  10541.205310
405    slipped fell   7870.520839
199        hit head   6592.746026
327      pain lower   4396.745014
142        felt pop   3944.576642
387      sharp pain   3936.107885
250        job duty   3703.200964
332   pain shoulder   3635.329673
213    index finger   3630.831582
316       pain back   3586.162339
340     parking lot   3515.999601
144      felt sharp   3235.195480
19        back pain   3175.789320
206       hurt back   3154.442670
464   twisted ankle   2837.379517
292   middle finger   2833.257924
446     struck head   2752.275330
62   causing injury   2709.056688
81       cut finger   2653.776971

In [15]:
acct_tfidf_freq2.sort_values(by=['freq2'], ascending =True). head(20)

acct_features2       freq2
265      legal defense  209.695030
88         defense doh  210.110988
273         loss claim  222.340121
67     claim bilateral  223.253762
32   bilateral hearing  265.429083
224     involved motor  314.310485
68         claim filed  321.823117
497       work slipped  331.918186
203       hitting back  339.045815
166        getting car  343.184588
48            car felt  343.514265
8            back area  350.267954
187          head neck  350.704683
41        bucket paint  352.335065
308          onto knee  353.861102
26            bed felt  355.253683
9            back felt  359.624369
483   walking customer  361.546004
49         car slipped  361.629200
459         truck fell  364.017093

In [16]:
inj_features2 = inj_vectorizer2.get_feature_names()
inj_features2

['2nd degree',
 '3rd finger',
 '4th finger',
 '5th finger',
 'allergic reaction',
 'ankle contusion',
 'ankle foot',
 'ankle fracture',
 'ankle knee',
 'ankle pain',
 'ankle sprain',
 'ankle strain',
 'area arm',
 'area back',
 'arm contusion',
 'arm elbow',
 'arm hand',
 'arm leg',
 'arm neck',
 'arm pain',
 'arm shoulder',
 'arm strain',
 'arm wrist',
 'asbestos exposure',
 'available coding',
 'back area',
 'back arm',
 'back contusion',
 'back hand',
 'back head',
 'back hip',
 'back injury',
 'back knee',
 'back leg',
 'back lifting',
 'back lower',
 'back muscle',
 'back neck',
 'back pain',
 'back rt',
 'back shoulder',
 'back side',
 'back sprain',
 'back strain',
 'back unknown',
 'back upper',
 'bee sting',
 'big toe',
 'bilateral hand',
 'bilateral hearing',
 'bilateral knee',
 'bilateral shoulder',
 'bilateral wrist',
 'bite drug',
 'blood pressure',
 'body eye',
 'body part',
 'box cutter',
 'box felt',
 'bump head',
 'burn hand',
 'carpal tunnel',
 'cart felt',
 'cervical

In [17]:
inj_freq2 = np.sum(inj_tfidf2, axis=0)
inj_tfidf_freq2 = pd.DataFrame({'inj_features2': inj_features2, 
                               'freq2': inj_freq2}) 
inj_tfidf_freq2.sort_values(by=['freq2'], ascending = False). head(20)

inj_features2         freq2
257       lower back  13595.508073
43       back strain   5966.080279
38         back pain   5885.990747
254         low back   3837.209195
188     index finger   3613.239194
314       pain lower   3595.288407
387  shoulder strain   3161.419653
469   unknown injury   3074.059016
280    middle finger   2939.437569
320    pain shoulder   2898.296516
423     strain lower   2683.101522
385    shoulder pain   2407.348683
311        pain knee   2351.377027
328       pain wrist   2153.581125
224      knee strain   2122.816128
220        knee pain   2114.992752
259        lower leg   2083.962226
350      ring finger   1945.859468
483       upper back   1829.785459
10      ankle sprain   1756.001547

In [18]:
# export result
filename = 'acct_tfidf_freq_unigram.csv'
acct_tfidf_freq1.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'acct_tfidf_freq_bigram.csv'
acct_tfidf_freq2.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_freq_unigram.csv'
inj_tfidf_freq1.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_freq_bigram.csv'
inj_tfidf_freq2.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

In [19]:
# np.save('acct_tfidf_mtx_unigram.npy', acct_tfidf1)
# np.save('acct_tfidf_mtx_bigram.npy', acct_tfidf2)
# np.save('inj_tfidf_mtx_unigram.npy', inj_tfidf1)
# np.save('inj_tfidf_mtx_bigram.npy', inj_tfidf2)

### Analysis by Target Response

In [41]:
acct_cleaned_hc = accident_cleaned[df['deEscInd3000'] == 0]
acct_cleaned_lc = accident_cleaned[df['deEscInd3000'] == 1]
inj_cleaned_hc = injury_cleaned[df['deEscInd3000'] == 0]
inj_cleaned_lc = injury_cleaned[df['deEscInd3000'] == 1]

In [42]:
print(acct_cleaned_hc.shape)
print(acct_cleaned_lc.shape)

print(inj_cleaned_hc.shape)
print(inj_cleaned_lc.shape)

(147251,)
(397278,)
(147251,)
(397278,)


#### Unigram Analysis

In [43]:
start_time = time.time()

acct_vectorizer1_hc = TfidfVectorizer(max_features=1000)
acct_vectorizer1_lc = TfidfVectorizer(max_features=1000)
inj_vectorizer1_hc = TfidfVectorizer(max_features=1000)
inj_vectorizer1_lc = TfidfVectorizer(max_features=1000)

acct_tfidf1_hc = acct_vectorizer1_hc.fit_transform(acct_cleaned_hc).toarray()
acct_tfidf1_lc = acct_vectorizer1_lc.fit_transform(acct_cleaned_lc).toarray()
inj_tfidf1_hc = inj_vectorizer1_hc.fit_transform(inj_cleaned_hc).toarray()
inj_tfidf1_lc = inj_vectorizer1_lc.fit_transform(inj_cleaned_lc).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 15.007398843765259 seconds ---


##### Accident Notes

In [44]:
acct_features1_hc = acct_vectorizer1_hc.get_feature_names()
acct_freq1_hc = np.sum(acct_tfidf1_hc, axis=0)
acct_tfidf_freq1_hc = pd.DataFrame({'acct_features1': acct_features1_hc, 
                               'freq1_hc': acct_freq1_hc}) 
acct_tfidf_freq1_hc.sort_values(by=['freq1_hc'], ascending = False). head(20)

acct_features1     freq1_hc
298           fell  5679.060785
48            back  5330.147234
589           pain  5223.906095
299           felt  4643.461057
437             iw  4069.068480
786        slipped  3883.919978
766       shoulder  3458.082302
451           knee  3433.045816
481        lifting  3109.632453
377           hand  2972.449616
963        walking  2852.736579
945        unknown  2512.084115
425         injury  2481.591460
142        causing  2392.498020
505          lower  2358.908617
319           foot  2354.063703
925          truck  2309.231319
398            hit  2303.292111
93             box  2152.926556
818          state  2145.108006

In [45]:
acct_features1_lc = acct_vectorizer1_lc.get_feature_names()
acct_freq1_lc = np.sum(acct_tfidf1_lc, axis=0)
acct_tfidf_freq1_lc = pd.DataFrame({'acct_features1': acct_features1_lc, 
                               'freq1_lc': acct_freq1_lc}) 
acct_tfidf_freq1_lc.sort_values(by=['freq1_lc'], ascending = False). head(20)

acct_features1      freq1_lc
53            back  13384.381147
300           fell  12594.437922
572           pain  12102.778644
301           felt  10692.624240
428             iw  10235.698101
372           hand   8778.042576
778        slipped   8530.647413
394            hit   7505.212175
959        walking   7054.959722
103            box   6884.617629
467        lifting   6772.661035
378           head   6602.745871
305         finger   6544.315900
438           knee   6522.535674
219            cut   6229.617058
941        unknown   5952.695860
318           foot   5934.058190
489          lower   5660.050814
153        causing   5573.009801
751       shoulder   5558.578874

In [46]:
acct_diff_uni = acct_tfidf_freq1_lc[~acct_tfidf_freq1_lc['acct_features1'].isin(acct_tfidf_freq1_hc['acct_features1'])]
acct_diff_uni.sort_values(by=['freq1_lc'], ascending = False)

acct_features1     freq1_lc
728        seizure  1131.324972
293        fainted   862.021214
661           rash   815.203179
667       reaction   776.492354
45          attack   728.673241
86          bitten   709.659994
23        allergic   683.373359
794       splashed   656.156975
85            bite   654.661321
842          stung   637.015187
380         headed   612.013997
111      breathing   595.558635
110         breath   547.036465
747     shoplifter   499.283228
383          heart   497.312873
767           skin   472.467790
73             bee   470.289267
770         sliced   462.975512
722        scraped   435.112654
792         spider   427.422896
109      breakroom   426.419722
723      scratched   422.578873
532          mulch   401.208517
619          poked   401.159076
411            ill   392.686200
822       sticking   391.779761
421         insect   391.234859
757           sick   389.422907
632    preexisting   379.231957
749      shortness   373.903238
..             ...          ...
206        couldnt   236.099815
662          razor   235.555199
846          sugar   234.507598
235     difficulty   232.446771
102           bowl   231.983082
699         robert   231.418722
856          swell   230.326681
31           apart   230.279678
721       scanning   227.838052
501           mark   227.799674
306       finished   226.845350
692       returned   226.091987
809         staple   225.290670
681         rental   225.122498
821          stick   224.624055
348         giving   224.092431
517          minor   223.408058
252          drink   220.909059
968           weak   219.850005
185           cold   218.062597
731         server   217.462318
587          patio   217.277301
168      christmas   215.580648
999      yesterday   212.853764
512        michael   212.347775
713           safe   211.812040
9             able   211.524823
35           apron   206.536488
961         wanted   205.360634
101        bounced   201.861909

[141 rows x 2 columns]

In [47]:
acct_diff_uni.shape #there are 140 unigram words that tend to appear in low cost claims only

(141, 2)

In [48]:
acct_comp_uni = pd.merge(acct_tfidf_freq1_lc, acct_tfidf_freq1_hc, on = 'acct_features1', how = 'inner')
acct_comp_uni['freq1_lc_avg'] = acct_comp_uni['freq1_lc']/acct_tfidf_freq1_lc.shape[0] 
acct_comp_uni['freq1_hc_avg'] = acct_comp_uni['freq1_hc']/acct_tfidf_freq1_hc.shape[0] 
acct_comp_uni['freq_diff'] = np.abs(acct_comp_uni['freq1_lc_avg'] - acct_comp_uni['freq1_hc_avg']) 
acct_comp_uni.sort_values(by=['freq_diff'], ascending = False).head(100)

acct_features1      freq1_lc     freq1_hc  freq1_lc_std  freq1_hc_std  \
38            back  13384.381147  5330.147234     13.384381      5.330147   
253           fell  12594.437922  5679.060785     12.594438      5.679061   
497           pain  12102.778644  5223.906095     12.102779      5.223906   
368             iw  10235.698101  4069.068480     10.235698      4.069068   
254           felt  10692.624240  4643.461057     10.692624      4.643461   
320           hand   8778.042576  2972.449616      8.778043      2.972450   
180            cut   6229.617058   916.844188      6.229617      0.916844   
339            hit   7505.212175  2303.292111      7.505212      2.303292   
258         finger   6544.315900  1703.344958      6.544316      1.703345   
80             box   6884.617629  2152.926556      6.884618      2.152927   
326           head   6602.745871  1927.680007      6.602746      1.927680   
667        slipped   8530.647413  3883.919978      8.530647      3.883920   
823        walking   7054.959722  2852.736579      7.054960      2.852737   
179       customer   5014.523683   966.406284      5.014524      0.966406   
404        lifting   6772.661035  3109.632453      6.772661      3.109632   
269           foot   5934.058190  2354.063703      5.934058      2.354064   
245            eye   3806.486442   306.137624      3.806486      0.306138   
849        working   5244.060150  1778.242070      5.244060      1.778242   
806        unknown   5952.695860  2512.084115      5.952696      2.512084   
111           cart   4776.485021  1408.786400      4.776485      1.408786   
424          lower   5660.050814  2358.908617      5.660051      2.358909   
204           door   4858.803619  1615.281993      4.858804      1.615282   
855          wrist   5189.341959  2007.655518      5.189342      2.007656   
121        causing   5573.009801  2392.498020      5.573010      2.392498   
499         pallet   4755.212365  1600.401635      4.755212      1.600402   
377           knee   6522.535674  3433.045816      6.522536      3.433046   
846           work   4821.361956  1866.384420      4.821362      1.866384   
266          floor   4968.643829  2050.709138      4.968644      2.050709   
302            got   4172.325181  1254.394887      4.172325      1.254395   
718         struck   4425.876463  1666.329636      4.425876      1.666330   
..             ...           ...          ...           ...           ...   
673      something   2142.769794   620.622640      2.142770      0.620623   
106            car   2703.790206  1190.548029      2.703790      1.190548   
384         ladder   3511.274672  2011.358122      3.511275      2.011358   
636          sharp   2386.784766   896.363485      2.386785      0.896363   
638          shelf   2043.299515   561.249806      2.043300      0.561250   
613           room   2075.757230   635.297655      2.075757      0.635298   
714       strained   2415.086031   988.321078      2.415086      0.988321   
511        patient   2243.631362   817.975396      2.243631      0.817975   
828          water   2222.992204   802.819387      2.222992      0.802819   
217            due   2647.047575  1230.424629      2.647048      1.230425   
220           duty   2866.188640  1475.206606      2.866189      1.475207   
358          index   1714.819146   326.420863      1.714819      0.326421   
483            one   1942.122795   563.442724      1.942123      0.563443   
66             bit   1541.235324   166.925103      1.541235      0.166925   
354            ice   2706.916288  1340.448528      2.706916      1.340449   
246           face   1849.449993   484.528459      1.849450      0.484528   
566        pushing   2337.441227   987.419950      2.337441      0.987420   
223          elbow   2558.653372  1209.408916      2.558653      1.209409   
445         middle   1780.155444   443.793132      1.780155      0.443793   
787         trying   2092.751278   759.677584      2.092751      0.759678   


##### Injury Notes

In [49]:
inj_features1_hc = inj_vectorizer1_hc.get_feature_names()
inj_freq1_hc = np.sum(inj_tfidf1_hc, axis=0)
inj_tfidf_freq1_hc = pd.DataFrame({'inj_features1': inj_features1_hc, 
                               'freq1_hc': inj_freq1_hc}) 
inj_tfidf_freq1_hc.sort_values(by=['freq1_hc'], ascending = False). head(20)

inj_features1      freq1_hc
950                 unknown  41388.538846
600                    pain   8532.151114
843                  strain   7735.492310
59                     back   7317.707329
772                shoulder   6053.935167
466                    knee   5633.585431
514                   lower   5097.947705
811  sprainstraintearjammed   4010.332237
444                  injury   3520.710895
996                   wrist   3431.027183
388                    hand   3310.614964
206               contusion   3125.459014
354                fracture   2902.962166
35                    ankle   2746.244640
561                    neck   2561.528936
41                      arm   2556.282631
808                  sprain   2520.085157
393                    head   2291.368164
730                      rt   2164.546134
333                  finger   2141.681663

In [50]:
inj_features1_lc = inj_vectorizer1_lc.get_feature_names()
inj_freq1_lc = np.sum(inj_tfidf1_lc, axis=0)
inj_tfidf_freq1_lc = pd.DataFrame({'inj_features1': inj_features1_lc, 
                               'freq1_lc': inj_freq1_lc}) 
inj_tfidf_freq1_lc.sort_values(by=['freq1_lc'], ascending = False). head(20)

inj_features1       freq1_lc
947                 unknown  100751.360722
799  sprainstraintearjammed   24030.640030
109         bruisecontusion   18255.629968
600                    pain   16878.376657
835                  strain   14878.485206
47                     back   14522.324226
514                   lower   10182.316923
467                    knee    9273.076345
196               contusion    8997.555969
384                    hand    8864.626939
477    lacerationcutscratch    8273.008384
760                shoulder    8026.380880
475              laceration    7742.754238
996                   wrist    7461.317185
441                  injury    7333.702640
323                  finger    6959.864986
389                    head    6545.709047
29                    ankle    5673.612012
796                  sprain    5095.851670
335                    foot    4859.988868

In [51]:
inj_diff_uni = inj_tfidf_freq1_lc[~inj_tfidf_freq1_lc['inj_features1'].isin(inj_tfidf_freq1_hc['inj_features1'])]
inj_diff_uni.sort_values(by=['freq1_lc'], ascending = False)

inj_features1     freq1_lc
304       eyedustdebris  1173.252306
842       strokeseizure  1073.942905
275                drug   623.543831
781               snake   613.828665
241          dermatitis   452.664931
850             sunburn   282.338095
825               sting   267.938930
694             redness   242.125435
64                  bee   212.593280
324          fingernail   207.023902
227              cutter   198.119311
476       lacerationcut   186.705020
232              debris   180.106143
202             corneal   170.858967
113                 bug   165.616248
790              spider   164.790572
773              sliced   148.554012
793            splashed   146.581765
871                  tb   139.841095
737           scratched   139.640432
567         needlestick   133.402285
608               panic   131.534977
84              blister   123.522780
576                none   122.368970
654            pregnant   121.526850
859  swellingskinjoints   121.000000
497         lightheaded   119.675270
308               faint   119.616006
639              poison   110.587266
452             itching   108.848019
..                  ...          ...
217               cream    50.775882
267               dough    50.745079
791             spilled    50.602325
322             filling    50.151556
151            checking    49.873345
714              ripped    49.858244
153              cheese    49.710938
898                tile    49.613216
783                soda    49.502604
857               swell    48.633062
738               screw    48.490780
315                fast    48.085594
814               steam    47.931462
682              raised    47.392935
821            sticking    47.347155
12                 able    47.136414
466             kitchen    46.407563
148              change    46.364226
721                 row    46.211800
723              rubber    46.203198
607               panel    44.923948
37                asked    44.853372
882                  th    44.569939
982                wine    44.393957
895                 tie    44.055210
522                make    43.829992
957               valve    43.547344
409                hold    42.506134
902              tipped    42.413612
486             leaning    40.512701

[146 rows x 2 columns]

In [52]:
inj_diff_uni.shape #there are 148 unigram words that tend to appear in low cost claims only

(146, 2)

In [53]:
inj_comp_uni = pd.merge(inj_tfidf_freq1_lc, inj_tfidf_freq1_hc, on = 'inj_features1', how = 'inner')
inj_comp_uni['freq1_lc_avg'] = inj_comp_uni['freq1_lc']/inj_tfidf_freq1_lc.shape[0] 
inj_comp_uni['freq1_hc_avg'] = inj_comp_uni['freq1_hc']/inj_tfidf_freq1_hc.shape[0] 
inj_comp_uni['freq_diff'] = np.abs(inj_comp_uni['freq1_lc_avg'] - inj_comp_uni['freq1_hc_avg']) 
inj_comp_uni.sort_values(by=['freq_diff'], ascending = False).head(100)

inj_features1       freq1_lc      freq1_hc  freq1_lc_std  \
807                 unknown  100751.360722  41388.538846    100.751361   
685  sprainstraintearjammed   24030.640030   4010.332237     24.030640   
93          bruisecontusion   18255.629968   1848.514134     18.255630   
517                    pain   16878.376657   8532.151114     16.878377   
407    lacerationcutscratch    8273.008384    428.387201      8.273008   
43                     back   14522.324226   7317.707329     14.522324   
715                  strain   14878.485206   7735.492310     14.878485   
406              laceration    7742.754238   1856.047312      7.742754   
169               contusion    8997.555969   3125.459014      8.997556   
329                    hand    8864.626939   3310.614964      8.864627   
440                   lower   10182.316923   5097.947705     10.182317   
280                  finger    6959.864986   2141.681663      6.959865   
334                    head    6545.709047   2291.368164      6.545709   
850                   wrist    7461.317185   3431.027183      7.461317   
379                  injury    7333.702640   3520.710895      7.333703   
270       faintingdizziness    3729.372737     52.000000      3.729373   
399                    knee    9273.076345   5633.585431      9.273076   
265                     eye    3973.252903    445.012622      3.973253   
578                puncture    3465.168486    351.248917      3.465168   
26                    ankle    5673.612012   2746.244640      5.673612   
195                     cut    3483.158636    661.301897      3.483159   
288                    foot    4859.988868   2048.615924      4.859989   
760                   thumb    3769.277742   1080.298918      3.769278   
682                  sprain    5095.851670   2520.085157      5.095852   
623                      rt    4551.574787   2164.546134      4.551575   
30                      arm    4625.042919   2556.282631      4.625043   
654                shoulder    8026.380880   6053.935167      8.026381   
369                   index    2528.897171    622.579455      2.528897   
100                    burn    2254.610496    445.465538      2.254610   
656                    side    2965.260725   1186.550487      2.965261   
..                      ...            ...           ...           ...   
669                 slipped    1259.525200    565.546907      1.259525   
10                 abrasion     905.745807    223.410806      0.905746   
543                   pinky    1002.694446    323.399570      1.002694   
610             respiratory     732.380668     56.423928      0.732381   
364                 illness     692.528548     30.675088      0.692529   
347             herniagroin     918.852718    269.788044      0.918853   
637                 seizure     700.318429     51.751800      0.700318   
590                    rash     690.020342     44.163948      0.690020   
553               poisoning     675.661183     32.810891      0.675661   
706                  stitch     844.732116    217.864328      0.844732   
520                  pallet     985.284268    360.284739      0.985284   
614                     rib    1222.795390    607.818717      1.222795   
731                 swollen     902.085980    295.124179      0.902086   
716                strained    1298.253615    695.142928      1.298254   
525                    part    1258.470653    656.249225      1.258471   
751                   thigh     950.651950    354.704013      0.950652   
387              irritation     621.439446     38.171327      0.621439   
423                 lifting    1155.129901    580.457425      1.155130   
799                 twisted    1122.007665    547.714054      1.122008   
116                    cart     916.711142    344.228899      0.916711   
559                possible    1387.076455    821.059123      1.387076   
534                physical     647.510370     86.951997      0.647510   
94                

In [54]:
# export result
filename = 'acct_tfidf_hc_freq_unigram.csv'
acct_tfidf_freq1_hc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'acct_tfidf_lc_freq_unigram.csv'
acct_tfidf_freq1_lc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'acct_tfidf_diffbycost_unigram.csv'
acct_diff_uni.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_hc_freq_unigram.csv'
inj_tfidf_freq1_hc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_lc_freq_unigram.csv'
inj_tfidf_freq1_lc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_diffbycost_unigram.csv'
inj_diff_uni.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

#### Bigram Analysis

In [55]:
start_time = time.time()

acct_vectorizer2_hc = TfidfVectorizer(max_features=500, ngram_range=(2,2))
acct_vectorizer2_lc = TfidfVectorizer(max_features=500, ngram_range=(2,2))
inj_vectorizer2_hc = TfidfVectorizer(max_features=500, ngram_range=(2,2))
inj_vectorizer2_lc = TfidfVectorizer(max_features=500, ngram_range=(2,2))

acct_tfidf2_hc = acct_vectorizer2_hc.fit_transform(acct_cleaned_hc).toarray()
acct_tfidf2_lc = acct_vectorizer2_lc.fit_transform(acct_cleaned_lc).toarray()
inj_tfidf2_hc = inj_vectorizer2_hc.fit_transform(inj_cleaned_hc).toarray()
inj_tfidf2_lc = inj_vectorizer2_lc.fit_transform(inj_cleaned_lc).toarray()

print("--- %s seconds ---" % (time.time() - start_time))

--- 19.8027286529541 seconds ---


##### Accident Notes

In [56]:
acct_features2_hc = acct_vectorizer2_hc.get_feature_names()
acct_freq2_hc = np.sum(acct_tfidf2_hc, axis=0)
acct_tfidf_freq2_hc = pd.DataFrame({'acct_features2': acct_features2_hc, 
                               'freq2_hc': acct_freq2_hc}) 
acct_tfidf_freq2_hc.sort_values(by=['freq2_hc'], ascending = False). head(20)

acct_features2     freq2_hc
130          felt pain  4217.047762
275         lower back  3138.560215
400       slipped fell  2759.434328
132           felt pop  1576.392611
324      pain shoulder  1454.961062
319         pain lower  1336.264412
240           job duty  1228.109861
183           hit head  1198.927167
384         sharp pain  1157.450131
309          pain back  1057.913867
365  repetitive motion  1047.268547
134         felt sharp   975.739378
464       twisted knee   957.623620
332        parking lot   947.546647
55      causing injury   943.982188
17           back pain   923.031687
404        slipped ice   907.084553
53        causing fall   861.594956
453       tripped fell   852.516755
101        fell ground   838.551672

In [57]:
acct_features2_lc = acct_vectorizer2_lc.get_feature_names()
acct_freq2_lc = np.sum(acct_tfidf2_lc, axis=0)
acct_tfidf_freq2_lc = pd.DataFrame({'acct_features2': acct_features2_lc, 
                               'freq2_lc': acct_freq2_lc}) 
acct_tfidf_freq2_lc.sort_values(by=['freq2_lc'], ascending = False). head(20)

acct_features2     freq2_lc
146      felt pain  8657.929197
276     lower back  7384.626314
204       hit head  5275.103775
402   slipped fell  5124.754469
320     pain lower  3084.823126
219   index finger  2990.802354
385     sharp pain  2787.897028
335    parking lot  2553.080737
310      pain back  2533.139460
212      hurt back  2443.863550
249       job duty  2427.710094
148       felt pop  2381.573189
81      cut finger  2340.721155
284  middle finger  2307.594016
150     felt sharp  2257.095121
16       back pain  2243.915594
66      chest pain  2195.445057
442    struck head  2182.276029
324  pain shoulder  2154.157770
461  twisted ankle  2108.156419

In [58]:
acct_diff_bi = acct_tfidf_freq2_lc[~acct_tfidf_freq2_lc['acct_features2'].isin(acct_tfidf_freq2_hc['acct_features2'])]
acct_diff_bi.sort_values(by=['freq2_lc'], ascending = False)

acct_features2     freq2_lc
1     allergic reaction  1098.927423
266        light headed   989.548677
29              big toe   979.968572
306      outside garden   911.724407
83            cut index   867.133839
142          felt dizzy   833.513013
35           box cutter   820.713480
176             got eye   768.461171
113        feeling well   726.752445
254           knife cut   716.594584
5    assisting customer   709.851468
255       knife slipped   709.629544
157           flat cart   674.863472
43          bumped head   671.334735
199             hit eye   649.217990
390    shortness breath   604.003248
40           break room   602.834394
77        customer load   597.845898
267    loading customer   577.593288
94              dog bit   569.894632
216          hurt wrist   531.317157
158            flew eye   527.938543
384        service desk   509.530491
429        stepped nail   501.370105
79     customer vehicle   476.138222
382        safety knife   472.809888
45         came contact   464.735204
411        splashed eye   464.100792
145           felt like   456.037241
311          pain chest   455.784253
..                  ...          ...
490           went home   302.458742
33              box box   302.403556
144          felt light   301.633614
78       customer truck   300.349117
167        freight cart   300.181970
190          head metal   300.079256
303         opened door   298.929076
99             door hit   297.600580
491           went pick   295.318664
499           wrong way   294.746494
424         stated felt   294.570926
350           pop wrist   294.097417
46             came hit   293.575418
401     slipped cutting   292.799010
282            mid back   292.473370
34              box cut   287.866998
42         bucket paint   286.939598
178           got worse   285.356443
237          iw picking   284.559742
85               cut rt   282.298204
358         pulling box   280.829909
98           door frame   279.174282
289         muscle back   277.299533
383           said felt   267.684407
38          box slipped   265.111052
171         getting car   264.543697
7             back felt   261.927791
332       pallet pallet   258.364037
53            cart fell   256.109887
89           cutter cut   230.320880

[144 rows x 2 columns]

In [59]:
acct_comp_bi = pd.merge(acct_tfidf_freq2_lc, acct_tfidf_freq2_hc, on = 'acct_features2', how = 'inner')
acct_comp_bi['freq2_lc_avg'] = acct_comp_bi['freq2_lc']/acct_tfidf_freq2_lc.shape[0] 
acct_comp_bi['freq2_hc_avg'] = acct_comp_bi['freq2_hc']/acct_tfidf_freq2_hc.shape[0] 
acct_comp_bi['freq_diff'] = np.abs(acct_comp_bi['freq2_lc_avg'] - acct_comp_bi['freq2_hc_avg']) 
acct_comp_bi.sort_values(by=['freq_diff'], ascending = False).head(100)

acct_features2     freq2_lc     freq2_hc  freq2_lc_std  freq2_hc_std  \
92           felt pain  8657.929197  4217.047762     17.315858      8.434096   
190         lower back  7384.626314  3138.560215     14.769253      6.277120   
132           hit head  5275.103775  1198.927167     10.550208      2.397854   
144       index finger  2990.802354   607.625023      5.981605      1.215250   
281       slipped fell  5124.754469  2759.434328     10.249509      5.518869   
47          cut finger  2340.721155   273.528757      4.681442      0.547058   
39          chest pain  2195.445057   138.905349      4.390890      0.277811   
196      middle finger  2307.594016   503.464139      4.615188      1.006928   
48            cut hand  2062.654613   291.662543      4.125309      0.583325   
222         pain lower  3084.823126  1336.264412      6.169646      2.672529   
138          hurt back  2443.863550   698.624666      4.887727      1.397249   
312        struck head  2182.276029   541.333897      4.364552      1.082668   
272         sharp pain  2787.897028  1157.450131      5.575794      2.314900   
233        parking lot  2553.080737   947.546647      5.106161      1.895093   
213          pain back  2533.139460  1057.913867      5.066279      2.115828   
49           cut thumb  1562.415768   166.469923      3.124832      0.332940   
325      twisted ankle  2108.156419   731.226332      4.216313      1.462453   
72            fell hit  1850.829674   502.524700      3.701659      1.005049   
11           back pain  2243.915594   923.031687      4.487831      1.846063   
96          felt sharp  2257.095121   975.739378      4.514190      1.951479   
170           job duty  2427.710094  1228.109861      4.855420      2.456220   
127   helping customer  1357.459790   188.450537      2.714920      0.376901   
229         pain wrist  1770.362928   686.219842      3.540726      1.372440   
168         iw walking  1653.419402   605.693313      3.306839      1.211387   
261        ring finger  1382.167942   356.223391      2.764336      0.712447   
81           fell onto  1604.673562   662.734282      3.209347      1.325469   
306      strained back  1390.460760   478.579589      2.780922      0.957159   
181        lifting box  1523.681757   638.771539      3.047364      1.277543   
232        pallet jack  1480.057055   598.574709      2.960114      1.197149   
169         iw working  1301.206145   423.492269      2.602412      0.846985   
..                 ...          ...          ...           ...           ...   
191          lower leg   848.695231   275.739641      1.697390      0.551479   
350          wet floor   952.326451   400.036621      1.904653      0.800073   
128           hit back   799.096408   247.977557      1.598193      0.495955   
267            rt hand   853.395590   318.946793      1.706791      0.637894   
294    started hurting   759.652078   227.803774      1.519304      0.455608   
98         felt strain  1038.334692   509.032732      2.076669      1.018065   
20           bent pick   712.295402   192.231765      1.424591      0.384464   
116          got stuck   874.539505   355.766904      1.749079      0.711534   
274      shopping cart   667.566957   152.306238      1.335134      0.304612   
133           hit knee   811.137960   300.733912      1.622276      0.601468   
278          side head   636.364512   134.130211      1.272729      0.268260   
241         piece wood   676.824986   178.922419      1.353650      0.357845   
324      turned around   709.476670   212.937130      1.418953      0.425874   
77           fell knee   839.306435   348.488293      1.678613      0.696977   
137          hot water   636.114126   145.728193      1.272228      0.291456   
235  performing normal   922.973170   433.502350      1.845946      0.867005   
292    started feeling   632.924438   143.940162      1.265849      0.287880   
18          began feel   620.888330   138.659061      1.241777      0.277318   
184     

##### Injury Notes

In [60]:
inj_features2_hc = inj_vectorizer2_hc.get_feature_names()
inj_freq2_hc = np.sum(inj_tfidf2_hc, axis=0)
inj_tfidf_freq2_hc = pd.DataFrame({'inj_features2': inj_features2_hc, 
                               'freq2_hc': inj_freq2_hc}) 
inj_tfidf_freq2_hc.sort_values(by=['freq2_hc'], ascending = False). head(20)

inj_features2     freq2_hc
257       lower back  4564.149574
42         back pain  1953.565392
47       back strain  1932.230036
385  shoulder strain  1391.505939
254         low back  1357.399077
316    pain shoulder  1296.036466
309       pain lower  1233.795870
381    shoulder pain  1079.780064
466   unknown injury  1028.968044
306        pain knee   962.672889
418     strain lower   914.733562
228      knee strain   862.747283
224        knee pain   814.475365
189     index finger   798.191241
424  strain shoulder   778.752058
276    middle finger   704.832873
324       pain wrist   606.136372
173     hearing loss   604.701829
259        lower leg   587.943956
479        upper arm   584.131008

In [61]:
inj_features2_lc = inj_vectorizer2_lc.get_feature_names()
inj_freq2_lc = np.sum(inj_tfidf2_lc, axis=0)
inj_tfidf_freq2_lc = pd.DataFrame({'inj_features2': inj_features2_lc, 
                               'freq2_lc': inj_freq2_lc}) 
inj_tfidf_freq2_lc.sort_values(by=['freq2_lc'], ascending = False). head(20)

inj_features2     freq2_lc
252         lower back  8993.055266
41         back strain  4029.228464
36           back pain  3940.569331
189       index finger  2794.630424
249           low back  2472.100333
308         pain lower  2363.527398
276      middle finger  2223.756779
472     unknown injury  2038.496867
424       strain lower  1769.064500
384    shoulder strain  1765.549659
314      pain shoulder  1584.420145
322         pain wrist  1552.626829
174       heart attack  1538.208167
254          lower leg  1461.251738
140       foreign body  1442.698045
348        ring finger  1428.727558
6    allergic reaction  1422.631362
305          pain knee  1389.324199
382      shoulder pain  1325.720278
127  finger laceration  1310.392640

In [62]:
inj_diff_bi = inj_tfidf_freq2_lc[~inj_tfidf_freq2_lc['inj_features2'].isin(inj_tfidf_freq2_hc['inj_features2'])]
inj_diff_bi.sort_values(by=['freq2_lc'], ascending = False)

inj_features2     freq2_lc
174           heart attack  1538.208167
6        allergic reaction  1422.631362
346    respiratory illness   820.964266
53                body eye   787.556739
400             snake bite   641.112866
100          drug chemical   637.295725
50               bite drug   637.295725
334       poisoning insect   637.295725
207           insect snake   637.295725
88              cut finger   528.955566
94               cut thumb   504.307694
108            eye foreign   430.326197
99                dog bite   404.775804
91               cut index   367.948890
372       shortness breath   366.117680
246               loss ear   361.049472
354                 rt eye   358.432314
243           light headed   354.972971
394        skin irritation   353.064106
175        heat exhaustion   339.156231
290           needle stick   306.621501
209  irritation dermatitis   303.917742
97      dermatitis sunburn   303.386941
109             eye injury   300.993027
123       finger contusion   294.743489
58               bump head   292.000727
55              box cutter   284.121144
129        finger puncture   282.812481
450               top foot   282.424585
324              palm hand   272.869104
..                     ...          ...
447         thumb puncture   148.148042
1               2nd finger   148.107902
161            hand sprain   147.726497
446             thumb pain   146.415477
222      laceration finger   146.212336
144              got stuck   145.826072
495       wrist laceration   143.736729
34             back muscle   140.938113
76    contusion laceration   139.944074
216        knee laceration   139.848619
244          little finger   136.637732
444            thumb index   136.406341
333             poison ivy   135.711497
39              back spasm   134.751228
274              mid lower   134.080110
341         puncture index   133.469214
395            slipped cut   133.301893
160              hand ring   133.254372
442            team member   133.041244
158              hand palm   132.859848
339         pulling pallet   126.603924
345       requiring stitch   126.301596
82         contusion upper   120.726963
264              lt middle   119.879241
177             high blood   115.138294
229          laceration lt   114.732301
93                  cut rt   114.170911
263               lt lower   112.967032
406              sprain rt   109.819659
328              part body   108.578572

[124 rows x 2 columns]

In [64]:
inj_comp_bi = pd.merge(inj_tfidf_freq2_lc, inj_tfidf_freq2_hc, on = 'inj_features2', how = 'inner')
inj_comp_bi['freq2_lc_avg'] = inj_comp_bi['freq2_lc']/inj_tfidf_freq2_lc.shape[0] 
inj_comp_bi['freq2_hc_avg'] = inj_comp_bi['freq2_hc']/inj_tfidf_freq2_hc.shape[0] 

inj_comp_bi['freq_diff'] = np.abs(inj_comp_bi['freq2_lc_avg'] - inj_comp_bi['freq2_hc_avg']) 
inj_comp_bi.sort_values(by=['freq_diff'], ascending = False).head(100)

inj_features2     freq2_lc     freq2_hc  freq2_lc_std  freq2_hc_std  \
178          lower back  8993.055266  4564.149574     17.986111      9.128299   
35          back strain  4029.228464  1932.230036      8.058457      3.864460   
133        index finger  2794.630424   798.191241      5.589261      1.596382   
31            back pain  3940.569331  1953.565392      7.881139      3.907131   
196       middle finger  2223.756779   704.832873      4.447514      1.409666   
98         foreign body  1442.698045    89.525936      2.885396      0.179052   
225          pain lower  2363.527398  1233.795870      4.727055      2.467592   
175            low back  2472.100333  1357.399077      4.944201      2.714798   
89    finger laceration  1310.392640   276.095042      2.620785      0.552190   
350      unknown injury  2038.496867  1028.968044      4.076994      2.057936   
239          pain wrist  1552.626829   606.136372      3.105254      1.212273   
50           chest pain  1089.353509   148.159377      2.178707      0.296319   
131            hit head  1236.140443   309.164649      2.472281      0.618329   
254         ring finger  1428.727558   537.831707      2.857455      1.075663   
180           lower leg  1461.251738   587.943956      2.922503      1.175888   
117      head contusion  1253.826607   395.222794      2.507653      0.790446   
314        strain lower  1769.064500   914.733562      3.538129      1.829467   
7          ankle sprain  1297.208951   458.978879      2.594418      0.917958   
177           lower arm  1130.434671   355.793457      2.260869      0.711587   
362          upper back  1300.085794   529.576375      2.600172      1.059153   
125  herniagroin strain  1036.832912   294.000000      2.073666      0.588000   
333            top head   856.162021   133.558336      1.712324      0.267117   
245        pinky finger   986.047503   330.867519      1.972095      0.661735   
243     physical injury   709.318955    89.610142      1.418638      0.179220   
57       contusion head   914.731496   298.798710      1.829463      0.597597   
24            back head   952.282142   359.484099      1.904564      0.718968   
81            felt pain  1129.973430   538.463880      2.259947      1.076928   
375        wrist strain   916.603614   329.732177      1.833207      0.659464   
331    thumb laceration   677.368747    96.361154      1.354737      0.192722   
164    laceration thumb   664.892196   102.270736      1.329784      0.204541   
..                  ...          ...          ...           ...           ...   
232           pain side   487.692889   198.206260      0.975386      0.396413   
55       contusion foot   387.418744    98.220074      0.774837      0.196440   
43            body part   638.490457   349.916764      1.276981      0.699834   
231       pain shoulder  1584.420145  1296.036466      3.168840      2.592073   
56       contusion hand   421.928864   135.522722      0.843858      0.271045   
185             lt hand   417.060731   132.521429      0.834121      0.265043   
271            rt wrist   517.564061   237.631381      1.035128      0.475263   
246      pointer finger   342.300515    63.005512      0.684601      0.126011   
195         middle back   467.527445   189.308011      0.935055      0.378616   
216           pain foot   463.725632   185.742202      0.927451      0.371484   
212          pain ankle   486.391987   214.953888      0.972784      0.429908   
162   laceration middle   340.191759    69.980881      0.680384      0.139962   
264             rt knee   743.148962   474.270068      1.486298      0.948540   
207         neck strain   619.586683   351.451283      1.239173      0.702903   
305        sprain wrist   385.500344   120.544243      0.771001      0.241088   
347      unknown finger   363.017556    99.134549      0.726035      0.198269   
95            foot pain   390.438813   127.596773      0.780878      0.255194   
112           hand pain   451.209144   18

In [65]:
# export result
filename = 'acct_tfidf_hc_freq_bigram.csv'
acct_tfidf_freq2_hc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'acct_tfidf_lc_freq_bigram.csv'
acct_tfidf_freq2_lc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'acct_tfidf_diffbycost_bigram.csv'
acct_diff_bi.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_hc_freq_bigram.csv'
inj_tfidf_freq2_hc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_lc_freq_bigram.csv'
inj_tfidf_freq2_lc.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)

filename = 'inj_tfidf_diffbycost_bigram.csv'
inj_diff_bi.to_csv(path_or_buf= f'{bucket}/{filename}', 
                   index=False)